<img src="Figs/GEOS_logo.pdf" width="500"/>

# Generate G matrix for the joint inversion: 
## <font color=blue>"generate_Gmatrix.ipynb"</font>
#### Dec 31, 2021  <font color=red>(v. testing)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)

1. This code is a part of the joint inversion project (project4: joint inversion of GNSS and InSAR)
2. The G-matrix will be built using two types of basis functions : BC and FT 
3. And then the final G-matrix will be save in a file. 

In [1]:
########Import modules
import numpy as np
import pandas as pd
import scipy 
import sys
import os

In [2]:
######## How Many Basis Functions (FT #ofCells; BC #ofRotations)
HowManyBasisFunctions=np.loadtxt("HowMany.txt")
HowManyCell=int(HowManyBasisFunctions[1])
HowManyRot=int(HowManyBasisFunctions[2])

In [21]:
# Output files
outputFILE_G_matrix="G_matrix.out" 
outputFILE_hori_continuous="G_matrix_cont_hori.out"
outputFILE_vert_continuous="G_matrix_cont_vert.out"
outputFILE_strain="G_matrix_strain.out"

# `STEP 1:` **BUILD a data vector,  $\vec{d}$**

In [4]:
# load input files

# 1. Boundary velocity (on the boundary)
inputBC = "BC_input.gmt"  # Boundary Condition Velocity [mm/yr]
df_inputBC = pd.read_csv(inputBC, header = None, sep =' ')
df_inputBC.columns = ['lon','lat','ve','vn','se','sn','corr']
df_inputBC.loc[:,['ve']] = df_inputBC.loc[:,['ve']]  
df_inputBC.loc[:,['vn']] = df_inputBC.loc[:,['vn']]

# 2. GNSS data
inputGNSS = "GNSS_input.gmt"  # GNSS [mm/yr]
df_inputGNSS = pd.read_csv(inputGNSS, header = None, sep=r'(?:,|\s+)', comment='#', engine='python')
df_inputGNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
df_inputGNSS.loc[:,['ve']] = df_inputGNSS.loc[:,['ve']] 
df_inputGNSS.loc[:,['vn']] = df_inputGNSS.loc[:,['vn']]

# 2. InSAR Descending 
inputInSAR_D = "dsd_InSAR_input.dat" # InSAR DECS [mm/yr]
df_inputInSAR_D = pd.read_csv(inputInSAR_D, header = None, sep = ' ')
df_inputInSAR_D.columns = ['lon','lat','velo','Px','Py','Pz']  
df_inputInSAR_D.loc[:,['velo']] = df_inputInSAR_D.loc[:,['velo']]

# 3. InSAR Ascending 
inputInSAR_A = "asd_InSAR_input.dat" # InSAR ASCE [mm/yr]
df_inputInSAR_A = pd.read_csv(inputInSAR_A, header = None, sep = ' ')
df_inputInSAR_A.columns = ['lon','lat','velo','Px','Py','Pz'] 
df_inputInSAR_A.loc[:,['velo']] = df_inputInSAR_A.loc[:,['velo']]

<div class="alert alert-success">
<b>NOTE: the pointing vectors are from the perspective of the ground! NOT of the satellite </b> 
</div>

In [5]:
##################################################################################################################################
##################################################################################################################################
######################################################         GNSS         ######################################################    
##################################################################################################################################
##################################################################################################################################

# BUILD GNSS data vector along with coordinate information.
# The x components are first and then the y components of the GNSS velocity.
df_data_x_GNSS = df_inputGNSS.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y_GNSS = df_inputGNSS.iloc[:,[0,1,3]]  # saved vn data on the boundary
df_data_x_GNSS=df_data_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
df_data_y_GNSS=df_data_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change

# !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!
df_data_x_GNSS=df_data_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y_GNSS=df_data_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])

# MERGE two columns (n*2) into a column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices
framesGNSS=[df_data_x_GNSS,df_data_y_GNSS]
df_data_GNSS_all=pd.concat(framesGNSS,ignore_index=True) # merge the two dataFrames into one

# SAVE GNSS velocity separately
df_data_GNSS=df_data_GNSS_all.loc[:,['velo']]


##################################################################################################################################
##################################################################################################################################
######################################################       Boundary       ######################################################    
##################################################################################################################################
##################################################################################################################################

# BUILD a boundary condition data vector along with coordinate information.
# The x components are first and then the y components of the velocity.
df_data_x_BC = df_inputBC.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y_BC = df_inputBC.iloc[:,[0,1,3]]  # saved vn data on the boundary
df_data_x_BC=df_data_x_BC.rename(columns ={'ve': 'velo'}) #column name change
df_data_y_BC=df_data_y_BC.rename(columns ={'vn': 'velo'}) #column name change

# !! SORT_VALUES !! # lat (ascending) first, and then lon (ascending).
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!
df_data_x_BC=df_data_x_BC.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y_BC=df_data_y_BC.sort_values(['lat', 'lon'], ascending=[True, True])

# MERGE two columns (n*2) into a column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices
framesBC=[df_data_x_BC,df_data_y_BC]
df_data_BC_all=pd.concat(framesBC,ignore_index=True) # merge the two dataFrames into one

# SAVE BC velocity separately
df_data_BC=df_data_BC_all.loc[:,['velo']]


##################################################################################################################################
##################################################################################################################################
######################################################        InSAR         ######################################################    
##################################################################################################################################
##################################################################################################################################

# BUILD a InSAR data vector along with coordinate information.
# The rows of the InSAR data vector is in the order of descending-orbit data, and ascending-orbit data. 
# Track the pointing vector values together with the rate data for the G-matrix.

# !! SORT_VALUES !! # lat (ascending) first, and then lon (ascending).
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!
df_inputInSAR_D=df_inputInSAR_D.sort_values(['lat', 'lon'], ascending=[True, True])
df_inputInSAR_A=df_inputInSAR_A.sort_values(['lat', 'lon'], ascending=[True, True])

# MERGE two columns (n*2) into a column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices
framesInSAR=[df_inputInSAR_D,df_inputInSAR_A]
df_data_InSAR_all=pd.concat(framesInSAR,ignore_index=True) # merge the four dataFrames into one

# SAVE InSAR velocity separately
df_data_InSAR = df_data_InSAR_all.loc[:,['velo']]

In [6]:
# Merge the GNSS, InSAR, and BC data vectors into the final data vector
framesFinal = [df_data_GNSS, df_data_InSAR, df_data_BC]
df_data_total = pd.concat(framesFinal,ignore_index=True) # merge the two dataFrames into one

In [7]:
df_data_total.columns = ['data'] # DATA VECTOR [[GNSSx], [GNSSy], [InSAR_D], [InSAR_A], [BCx], [BCy]]

# `STEP 2:` **BUILD G-Matrix, $\bar{\bar{G}}$**

### `STEP 2-a :` Build G matrix parts related to Rotations for Boundary Conditions

In [8]:
df_G_BC_on_GNSS = pd.DataFrame(index = range(len(df_data_GNSS))) 
# Make a blank G matrix part related to Boundary Condition on GNSS points

df_G_BC_on_boundary= pd.DataFrame(index = range(len(df_data_BC))) 
# Make a blank G matrix part related to Boundary Condition on boundary points

df_G_BC_on_InSAR = pd.DataFrame(index = range(len(df_data_InSAR))) 
# Make a blank G matrix part related to Boundary Condition on InSAR points
# df_data_InSAR : velocity only
# df_data_InSAR_all : lon lat vel px py pz
df_px = df_data_InSAR_all.iloc[:,[3]]
df_py = df_data_InSAR_all.iloc[:,[4]]
df_pz = df_data_InSAR_all.iloc[:,[5]]


continuous_sample = np.loadtxt('./basis_functions_BC/vel_BC_x_001_continuous.gmt')
continuous_XandY=len(continuous_sample)*2
df_G_BC_continuous = pd.DataFrame(index = range(continuous_XandY)) 
# Make a blank G matrix part for continuous horizontal velocity
# NOTE: no vertical motions in the BC basis functions 

midpoints_sample = np.loadtxt('./basis_functions_BC/average_strain_BC_x_001_RECTANGULAR.out')
midpoints_XXandYYandXY = len(midpoints_sample)*3
df_G_BC_on_midpoints_strain =pd.DataFrame(index = range(midpoints_XXandYYandXY)) 
# Make a blank G matrix part for strain



for i in range(1,HowManyRot+1): 

    
##################################################################################################################################
##################################################################################################################################
######################################################         GNSS         ######################################################    
##################################################################################################################################
################################################################################################################################## 

    
    inputfile_xrot_GNSS = "./basis_functions_BC/vel_BC_x_"+str(f"{i:03}")+"_on_GNSS.gmt" # x-rot 
    inputfile_yrot_GNSS = "./basis_functions_BC/vel_BC_y_"+str(f"{i:03}")+"_on_GNSS.gmt" # y-rot 
    inputfile_zrot_GNSS = "./basis_functions_BC/vel_BC_z_"+str(f"{i:03}")+"_on_GNSS.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowManyRot, yrotHowManyRot, zrotHowManyRot}

    df_xrot_GNSS=pd.read_csv(inputfile_xrot_GNSS ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot_GNSS=pd.read_csv(inputfile_yrot_GNSS ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot_GNSS=pd.read_csv(inputfile_zrot_GNSS ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x_GNSS = df_xrot_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_xrot_y_GNSS = df_xrot_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_xrot_x_GNSS=df_xrot_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y_GNSS=df_xrot_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x_GNSS=df_xrot_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y_GNSS=df_xrot_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx_GNSS=[df_xrot_x_GNSS,df_xrot_y_GNSS]
    df_Gx_GNSS=pd.concat(frames_Gx_GNSS,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x_GNSS = df_yrot_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_yrot_y_GNSS = df_yrot_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_yrot_x_GNSS=df_yrot_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y_GNSS=df_yrot_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x_GNSS=df_yrot_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y_GNSS=df_yrot_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy_GNSS=[df_yrot_x_GNSS,df_yrot_y_GNSS]
    df_Gy_GNSS=pd.concat(frames_Gy_GNSS,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x_GNSS = df_zrot_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_zrot_y_GNSS = df_zrot_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_zrot_x_GNSS=df_zrot_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y_GNSS=df_zrot_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change
   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x_GNSS=df_zrot_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y_GNSS=df_zrot_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz_GNSS=[df_zrot_x_GNSS,df_zrot_y_GNSS]
    df_Gz_GNSS=pd.concat(frames_Gz_GNSS,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowManyRot) Gyrot(HowManyRot) Gzrot(HowManyRot)]
    
    df_G_BC_on_GNSS["G_xrot"+str(i)] = df_Gx_GNSS.loc[:,['velo']]
    df_G_BC_on_GNSS["G_yrot"+str(i)] = df_Gy_GNSS.loc[:,['velo']]
    df_G_BC_on_GNSS["G_zrot"+str(i)] = df_Gz_GNSS.loc[:,['velo']]
    
    

##################################################################################################################################
##################################################################################################################################
######################################################       Boundary       ######################################################    
##################################################################################################################################
##################################################################################################################################



    inputfile_xrot_on_boundary = "./basis_functions_BC/vel_BC_x_"+str(f"{i:03}")+"_on_boundary.gmt" # x-rot 
    inputfile_yrot_on_boundary = "./basis_functions_BC/vel_BC_y_"+str(f"{i:03}")+"_on_boundary.gmt" # y-rot 
    inputfile_zrot_on_boundary = "./basis_functions_BC/vel_BC_z_"+str(f"{i:03}")+"_on_boundary.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowManyRot, yrotHowManyRot, zrotHowManyRot}

    df_xrot_on_boundary=pd.read_csv(inputfile_xrot_on_boundary ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot_on_boundary=pd.read_csv(inputfile_yrot_on_boundary ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot_on_boundary=pd.read_csv(inputfile_zrot_on_boundary ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot_on_boundary.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot_on_boundary.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot_on_boundary.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x_on_boundary = df_xrot_on_boundary.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_xrot_y_on_boundary = df_xrot_on_boundary.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_xrot_x_on_boundary=df_xrot_x_on_boundary.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y_on_boundary=df_xrot_y_on_boundary.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x_on_boundary=df_xrot_x_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y_on_boundary=df_xrot_y_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx_on_boundary=[df_xrot_x_on_boundary,df_xrot_y_on_boundary]
    df_Gx_on_boundary=pd.concat(frames_Gx_on_boundary,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x_on_boundary = df_yrot_on_boundary.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_yrot_y_on_boundary = df_yrot_on_boundary.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_yrot_x_on_boundary=df_yrot_x_on_boundary.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y_on_boundary=df_yrot_y_on_boundary.rename(columns ={'vn': 'velo'}) #column name change


    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x_on_boundary=df_yrot_x_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y_on_boundary=df_yrot_y_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy_on_boundary=[df_yrot_x_on_boundary,df_yrot_y_on_boundary]
    df_Gy_on_boundary=pd.concat(frames_Gy_on_boundary,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x_on_boundary = df_zrot_on_boundary.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_zrot_y_on_boundary = df_zrot_on_boundary.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_zrot_x_on_boundary=df_zrot_x_on_boundary.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y_on_boundary=df_zrot_y_on_boundary.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x_on_boundary=df_zrot_x_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y_on_boundary=df_zrot_y_on_boundary.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz_on_boundary=[df_zrot_x_on_boundary,df_zrot_y_on_boundary]
    df_Gz_on_boundary=pd.concat(frames_Gz_on_boundary,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowManyRot) Gyrot(HowManyRot) Gzrot(HowManyRot)]
    
    df_G_BC_on_boundary["G_xrot"+str(i)] = df_Gx_on_boundary.loc[:,['velo']]
    df_G_BC_on_boundary["G_yrot"+str(i)] = df_Gy_on_boundary.loc[:,['velo']]
    df_G_BC_on_boundary["G_zrot"+str(i)] = df_Gz_on_boundary.loc[:,['velo']]

    
    
##################################################################################################################################
##################################################################################################################################
######################################################        InSAR         ######################################################    
##################################################################################################################################
##################################################################################################################################


    inputfile_xrot_InSAR = "./basis_functions_BC/vel_BC_x_"+str(f"{i:03}")+"_on_InSAR.gmt" # x-rot 
    inputfile_yrot_InSAR = "./basis_functions_BC/vel_BC_y_"+str(f"{i:03}")+"_on_InSAR.gmt" # y-rot 
    inputfile_zrot_InSAR = "./basis_functions_BC/vel_BC_z_"+str(f"{i:03}")+"_on_InSAR.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowManyRot, yrotHowManyRot, zrotHowManyRot}

    df_xrot_InSAR=pd.read_csv(inputfile_xrot_InSAR ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot_InSAR=pd.read_csv(inputfile_yrot_InSAR ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot_InSAR=pd.read_csv(inputfile_zrot_InSAR ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 
    # ve = Px
    # vn = Py
    # This is because later on ...
    # 've' will be multiplied by a dataFrame named 'Px'
    # 'vn' will be multiplied by a dataFrame named 'Py'
    # pandas does NOT allow to multiply (element-wise) DataFrames if they have different names

    df_xrot_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']  
    df_yrot_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_zrot_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']
    

    
# SORT THEM by the same order of the InSAR data,
# which was already sorted in the beginning of this code.
    df_xrot_InSAR=df_xrot_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_InSAR=df_yrot_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_InSAR=df_zrot_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])

# STACK the entire basis function responses vertically TWICE!
# InSAR data vector is made of TWO different data sets (Descending and Ascending Orbits)
# But the 2 data sets are sampled in the same coordinates. 
    frames_xrot_stack_InSAR = [df_xrot_InSAR,df_xrot_InSAR]
    df_xrot_stacked_InSAR=pd.concat(frames_xrot_stack_InSAR,ignore_index=True) # merge the two dataFrames into one    
    frames_yrot_stack_InSAR = [df_yrot_InSAR,df_yrot_InSAR]
    df_yrot_stacked_InSAR=pd.concat(frames_yrot_stack_InSAR,ignore_index=True) # merge the two dataFrames into one   
    frames_zrot_stack_InSAR = [df_zrot_InSAR,df_zrot_InSAR]
    df_zrot_stacked_InSAR=pd.concat(frames_zrot_stack_InSAR,ignore_index=True) # merge the two dataFrames into one
    
    
# # BUILD a column vector Gx (i)
    df_LOS_Gxrot_x_InSAR = df_xrot_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Gxrot_y_InSAR = df_xrot_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Gxrot_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gxrot_InSAR = df_LOS_Gxrot_x_InSAR + df_LOS_Gxrot_y_InSAR
    

    df_LOS_Gyrot_x_InSAR = df_yrot_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Gyrot_y_InSAR = df_yrot_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Gyrot_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gyrot_InSAR = df_LOS_Gyrot_x_InSAR + df_LOS_Gyrot_y_InSAR
    

    df_LOS_Gzrot_x_InSAR = df_zrot_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Gzrot_y_InSAR = df_zrot_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Gzrot_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gzrot_InSAR = df_LOS_Gzrot_x_InSAR + df_LOS_Gzrot_y_InSAR
    

# SAVE G-matrix
# df_G_FT_on_InSAR_hori = [(xrot1_x*px + xrot1_y*py), ... , (zrotHowManyRot_x*px + zrotHowManyRot_y*py)]

    
    df_G_BC_on_InSAR["GxrotXPx+GxrotYPy "+str(i)] = df_LOS_Gxrot_InSAR.loc[:,['Py']]
    df_G_BC_on_InSAR["GyrotXPx+GyrotYPy "+str(i)] = df_LOS_Gyrot_InSAR.loc[:,['Py']]
    df_G_BC_on_InSAR["GzrotXPx+GzrotYPy "+str(i)] = df_LOS_Gzrot_InSAR.loc[:,['Py']]

    
    
##################################################################################################################################
##################################################################################################################################
######################################################   Continuous Field   ######################################################    
##################################################################################################################################
##################################################################################################################################    

    inputfile_xrot_continuous = "./basis_functions_BC/vel_BC_x_"+str(f"{i:03}")+"_continuous.gmt" # x-rot 
    inputfile_yrot_continuous = "./basis_functions_BC/vel_BC_y_"+str(f"{i:03}")+"_continuous.gmt" # y-rot 
    inputfile_zrot_continuous = "./basis_functions_BC/vel_BC_z_"+str(f"{i:03}")+"_continuous.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowManyRot, yrotHowManyRot, zrotHowManyRot}

    df_xrot_continuous=pd.read_csv(inputfile_xrot_continuous ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot_continuous=pd.read_csv(inputfile_yrot_continuous ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot_continuous=pd.read_csv(inputfile_zrot_continuous ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x_continuous = df_xrot_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function 
    df_xrot_y_continuous = df_xrot_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function 

    df_xrot_x_continuous=df_xrot_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y_continuous=df_xrot_y_continuous.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x_continuous=df_xrot_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y_continuous=df_xrot_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx_continuous=[df_xrot_x_continuous,df_xrot_y_continuous]
    df_Gx_continuous=pd.concat(frames_Gx_continuous,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Gy (i)

    df_yrot_x_continuous = df_yrot_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function
    df_yrot_y_continuous = df_yrot_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_yrot_x_continuous=df_yrot_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y_continuous=df_yrot_y_continuous.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x_continuous=df_yrot_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y_continuous=df_yrot_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy_continuous=[df_yrot_x_continuous,df_yrot_y_continuous]
    df_Gy_continuous=pd.concat(frames_Gy_continuous,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x_continuous = df_zrot_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function 
    df_zrot_y_continuous = df_zrot_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_zrot_x_continuous=df_zrot_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y_continuous=df_zrot_y_continuous.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x_continuous=df_zrot_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y_continuous=df_zrot_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz_continuous=[df_zrot_x_continuous,df_zrot_y_continuous]
    df_Gz_continuous=pd.concat(frames_Gz_continuous,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowManyRot) Gyrot(HowManyRot) Gzrot(HowManyRot)]
    
    df_G_BC_continuous["G_xrot"+str(i)] = df_Gx_continuous.loc[:,['velo']]
    df_G_BC_continuous["G_yrot"+str(i)] = df_Gy_continuous.loc[:,['velo']]
    df_G_BC_continuous["G_zrot"+str(i)] = df_Gz_continuous.loc[:,['velo']]
    


#########STRAIN ###########

    inputfile_strain_xrot = "./basis_functions_BC/average_strain_BC_x_"+str(f"{i:03}")+"_RECTANGULAR.out" # x-rot 
    inputfile_strain_yrot = "./basis_functions_BC/average_strain_BC_y_"+str(f"{i:03}")+"_RECTANGULAR.out" # y-rot 
    inputfile_strain_zrot = "./basis_functions_BC/average_strain_BC_z_"+str(f"{i:03}")+"_RECTANGULAR.out" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}
    df_xrot_strain=pd.read_csv(inputfile_strain_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot_strain=pd.read_csv(inputfile_strain_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot_strain=pd.read_csv(inputfile_strain_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
# CHANGE the column names 
    df_xrot_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    df_yrot_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    df_zrot_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']

# BUILD a column vector Gx (i)
    df_xrot_exx = df_xrot_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints for strain
    df_xrot_eyy = df_xrot_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints for strain
    df_xrot_exy = df_xrot_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints for strain
    
    df_xrot_exx=df_xrot_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_xrot_eyy=df_xrot_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_xrot_exy=df_xrot_exy.rename(columns ={'exy': 'strain'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_exx=df_xrot_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_eyy=df_xrot_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_exy=df_xrot_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx_strain=[df_xrot_exx,df_xrot_eyy,df_xrot_exy]
    df_Gx_strain=pd.concat(frames_Gx_strain,ignore_index=True) # merge the three dataFrames into one
    
    
# BUILD a column vector Gy (i)
    df_yrot_exx = df_yrot_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints for strain
    df_yrot_eyy = df_yrot_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints for strain
    df_yrot_exy = df_yrot_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints for strain
    
    df_yrot_exx=df_yrot_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_yrot_eyy=df_yrot_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_yrot_exy=df_yrot_exy.rename(columns ={'exy': 'strain'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_exx=df_yrot_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_eyy=df_yrot_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_exy=df_yrot_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gy_strain=[df_yrot_exx,df_yrot_eyy,df_yrot_exy]
    df_Gy_strain=pd.concat(frames_Gy_strain,ignore_index=True) # merge the three dataFrames into one    
    
# BUILD a column vector Gz (i)
    df_zrot_exx = df_zrot_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints for strain
    df_zrot_eyy = df_zrot_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints for strain
    df_zrot_exy = df_zrot_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints for strain
    
    df_zrot_exx=df_zrot_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_zrot_eyy=df_zrot_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_zrot_exy=df_zrot_exy.rename(columns ={'exy': 'strain'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_exx=df_zrot_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_eyy=df_zrot_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_exy=df_zrot_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gz_strain=[df_zrot_exx,df_zrot_eyy,df_zrot_exy]
    df_Gz_strain=pd.concat(frames_Gz_strain,ignore_index=True) # merge the three dataFrames into one 
    
    
    df_G_BC_on_midpoints_strain["G_xrot"+str(i)] = df_Gx_strain.loc[:,['strain']]
    df_G_BC_on_midpoints_strain["G_yrot"+str(i)] = df_Gy_strain.loc[:,['strain']]
    df_G_BC_on_midpoints_strain["G_zrot"+str(i)] = df_Gz_strain.loc[:,['strain']]


/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2296519195.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_G_BC_on_GNSS["G_yrot"+str(i)] = df_Gy_GNSS.loc[:,['velo']]
/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2296519195.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_G_BC_on_GNSS["G_zrot"+str(i)] = df_Gz_GNSS.loc[:,['velo']]
/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2296519195.py:240: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

### `STEP 2-b :` Build G matrix part related to Force Terms

In [9]:
df_G_FT_on_GNSS_eij = pd.DataFrame(index = range(len(df_data_GNSS))) 
df_G_FT_on_GNSS_ezz = pd.DataFrame(index = range(len(df_data_GNSS)))
# Make a blank G matrix part related to Boundary Condition on GNSS data points

df_G_FT_on_InSAR_hori = pd.DataFrame(index = range(len(df_data_InSAR))) 
df_G_FT_on_InSAR_vert = pd.DataFrame(index = range(len(df_data_InSAR))) 
# Make a blank G matrix part related to Force Terms on InSAR data points
# df_data_InSAR : velocity only
# df_data_InSAR_all : lon lat vel px py pz
df_px = df_data_InSAR_all.iloc[:,[3]]
df_py = df_data_InSAR_all.iloc[:,[4]]
df_pz = df_data_InSAR_all.iloc[:,[5]]


continuous_sample = np.loadtxt('./basis_functions_FT/vel_hori_FT_1_1_continuous.gmt')
continuous_XandY=len(continuous_sample)*2
continuous_Z=len(continuous_sample)

df_G_FT_continuous_eij = pd.DataFrame(index = range(continuous_XandY)) 
df_G_FT_continuous_ezz = pd.DataFrame(index = range(continuous_XandY)) 
df_G_FT_continuous_zzz = pd.DataFrame(index = range(continuous_Z))
# Make a blank G matrix part for velocity

midpoints_sample = np.loadtxt('./basis_functions_FT/average_strain_FT_1_1_RECTANGULAR.out')
midpoints_XXandYYandXY = len(midpoints_sample)*3

df_G_FT_on_midpoints_eij =pd.DataFrame(index = range(midpoints_XXandYYandXY))
df_G_FT_on_midpoints_ezz =pd.DataFrame(index = range(midpoints_XXandYYandXY))
# Make a blank G matrix part for strain


for i in range(1,HowManyCell+1): 

##################################################################################################################################
##################################################################################################################################
######################################################         GNSS         ######################################################    
##################################################################################################################################
##################################################################################################################################     
    
    inputfile_exx_GNSS = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_1"+"_on_GNSS.gmt" #exx horizontal
    inputfile_eyy_GNSS = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_2"+"_on_GNSS.gmt" #eyy horizontal
    inputfile_exy_GNSS = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_3"+"_on_GNSS.gmt" #exy horizontal 
    inputfile_ezz_GNSS = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_4"+"_on_GNSS.gmt" # z  horizontal
    
## READ files into two separate structures in the following orders:
## 1st stru = {exx1, eyy1, exy1, ..., exxHowManyCell, eyyHowManyCell, exyHowManyCell} 
## 2nd stru = {z1 .. zHowManyCell}
##
## And then merge these two structures into one in the order of ...
##            {exx1, eyy1, exy1, ..., exxHowManyCell, eyyHowManyCell, exyHowManyCell, z1 .. zHowManyCell}


    df_exx_GNSS=pd.read_csv(inputfile_exx_GNSS, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy_GNSS=pd.read_csv(inputfile_eyy_GNSS, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy_GNSS=pd.read_csv(inputfile_exy_GNSS, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz_GNSS=pd.read_csv(inputfile_ezz_GNSS, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')

# CHANGE the column names 

    df_exx_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_eyy_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_exy_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_ezz_GNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gexx (i)

    df_exx_x_GNSS = df_exx_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_exx_y_GNSS = df_exx_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_exx_x_GNSS=df_exx_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_exx_y_GNSS=df_exx_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_x_GNSS=df_exx_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_y_GNSS=df_exx_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx_GNSS=[df_exx_x_GNSS,df_exx_y_GNSS]
    df_Gexx_GNSS=pd.concat(frames_Gexx_GNSS,ignore_index=True) # merge the two dataFrames into one

# BUILD a column vector Geyy (i)

    df_eyy_x_GNSS = df_eyy_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_eyy_y_GNSS = df_eyy_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_eyy_x_GNSS=df_eyy_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_eyy_y_GNSS=df_eyy_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_x_GNSS=df_eyy_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_y_GNSS=df_eyy_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Geyy_GNSS=[df_eyy_x_GNSS,df_eyy_y_GNSS]
    df_Geyy_GNSS=pd.concat(frames_Geyy_GNSS,ignore_index=True) # merge the two dataFrames into one
    
# BUILD a column vector Gexy (i)

    df_exy_x_GNSS = df_exy_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_exy_y_GNSS = df_exy_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_exy_x_GNSS=df_exy_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_exy_y_GNSS=df_exy_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_x_GNSS=df_exy_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_y_GNSS=df_exy_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gexy_GNSS=[df_exy_x_GNSS,df_exy_y_GNSS]
    df_Gexy_GNSS=pd.concat(frames_Gexy_GNSS,ignore_index=True) # merge the two dataFrames into one


# BUILD a column vector G_ezz (i)

    df_ezz_x_GNSS = df_ezz_GNSS.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS data points
    df_ezz_y_GNSS = df_ezz_GNSS.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS data points

    df_ezz_x_GNSS=df_ezz_x_GNSS.rename(columns ={'ve': 'velo'}) #column name change
    df_ezz_y_GNSS=df_ezz_y_GNSS.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_x_GNSS=df_ezz_x_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_y_GNSS=df_ezz_y_GNSS.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gezz_GNSS=[df_ezz_x_GNSS,df_ezz_y_GNSS]
    df_Gezz_GNSS=pd.concat(frames_Gezz_GNSS,ignore_index=True) # merge the two dataFrames into one (vertically)
    
    
    
# SAVE a part of G-matrix (as in two different structures and then they will be merged later)

    # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowManyCell) Geyy(HowManyCell) Gexy(HowManyCell)]   
    df_G_FT_on_GNSS_eij["G_exx"+str(i)] = df_Gexx_GNSS.loc[:,['velo']]
    df_G_FT_on_GNSS_eij["G_eyy"+str(i)] = df_Geyy_GNSS.loc[:,['velo']]
    df_G_FT_on_GNSS_eij["G_exy"+str(i)] = df_Gexy_GNSS.loc[:,['velo']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowManyCell)] 
    df_G_FT_on_GNSS_ezz["G_ezz"+str(i)] = df_Gezz_GNSS.loc[:,['velo']]

    
##################################################################################################################################
##################################################################################################################################
######################################################       Boundary       ######################################################    
##################################################################################################################################
##################################################################################################################################
    
# A np.zeros matrix will be generated at the end of this block of the code
    
##################################################################################################################################
##################################################################################################################################
######################################################        InSAR         ######################################################    
##################################################################################################################################
##################################################################################################################################
    
    inputfile_exx_InSAR = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_1"+"_on_InSAR.gmt" #exx horizontal
    inputfile_eyy_InSAR = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_2"+"_on_InSAR.gmt" #eyy horizontal
    inputfile_exy_InSAR = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_3"+"_on_InSAR.gmt" #exy horizontal 
    inputfile_ezz_InSAR = "./basis_functions_FT/vel_vert_FT_"+str(i)+"_4"+"_on_InSAR.gmt" # z  vertical
    
## READ files into two separate structures in the following orders:
## 1st stru = {exx1_x*px + exx1_y*py, eyy1_x*px + eyy1_y*py , exy1_x*px + exy1_y*px, ..., 
##             eyyHowManyCell_x*px + eyyHowManyCell_y*py, exyHowManyCell_x*px + exyHowManyCell_y*py} 
## 2nd stru = {ezz1_z*pz, ezz2_z*pz, ..., ezzHowManyCell_z*pz}
##
## And then merge these two structures into one in the order of ...
##            {exx1_x*px + exx1_y*py,...,exyHowManyCell_x*px + exyHowManyCell_y*py,ezz1_z*pz, ..., ezzHowManyCell_z*pz}


    df_exx_InSAR=pd.read_csv(inputfile_exx_InSAR, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy_InSAR=pd.read_csv(inputfile_eyy_InSAR, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy_InSAR=pd.read_csv(inputfile_exy_InSAR, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz_InSAR=pd.read_csv(inputfile_ezz_InSAR, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')

# CHANGE the column names 

    # ve = Px
    # vn = Py
    # vz = Pz 
    # This is because later on ...
    # 've' will be multiplied by a dataFrame named 'Px'
    # 'vn' will be multiplied by a dataFrame named 'Py'
    # 'vz' will be multiplied by a dataFrame named 'Pz'
    # pandas does NOT allow to multiply (element wise) two different DataFrames if they have different names
    
    df_exx_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_eyy_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_exy_InSAR.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_ezz_InSAR.columns = ['lon','lat','Pz']

# SORT THEM by the same order of the InSAR data,
# which was already sorted in the beginning of this code.
    df_exx_InSAR=df_exx_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_InSAR=df_eyy_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_InSAR=df_exy_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_InSAR=df_ezz_InSAR.sort_values(['lat', 'lon'], ascending=[True, True])

# STACK the entire basis function responses vertically TWICE!
# InSAR data vector is made of TWO different data sets (Descending and Ascending Orbits)
# But the 2 data sets are sampled in the same coordinates. 
    frames_exx_stack_InSAR = [df_exx_InSAR,df_exx_InSAR] # Two insar data sets!
    df_exx_stacked_InSAR=pd.concat(frames_exx_stack_InSAR,ignore_index=True) 
    # merge the two dataFrames into one

    frames_eyy_stack_InSAR = [df_eyy_InSAR,df_eyy_InSAR] # Two insar data sets!
    df_eyy_stacked_InSAR=pd.concat(frames_eyy_stack_InSAR,ignore_index=True) 
    # merge the two dataFrames into one    

    frames_exy_stack_InSAR = [df_exy_InSAR,df_exy_InSAR] # Two insar data sets!
    df_exy_stacked_InSAR=pd.concat(frames_exy_stack_InSAR,ignore_index=True) 
    # merge the two dataFrames into one

    frames_ezz_stack_InSAR = [df_ezz_InSAR,df_ezz_InSAR] # Two insar data sets!
    df_ezz_stacked_InSAR=pd.concat(frames_ezz_stack_InSAR,ignore_index=True) 
    # merge the two dataFrames into one

    
    
# # BUILD a column vector Gx (i)
    df_LOS_Gexx_x_InSAR = df_exx_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Gexx_y_InSAR = df_exx_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Gexx_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gexx_InSAR = df_LOS_Gexx_x_InSAR + df_LOS_Gexx_y_InSAR
    
    
    df_LOS_Geyy_x_InSAR = df_eyy_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Geyy_y_InSAR = df_eyy_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Geyy_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Geyy_InSAR = df_LOS_Geyy_x_InSAR + df_LOS_Geyy_y_InSAR
    
    
    df_LOS_Gexy_x_InSAR = df_exy_stacked_InSAR.loc[:,['Px']] * df_px 
    df_LOS_Gexy_y_InSAR = df_exy_stacked_InSAR.loc[:,['Py']] * df_py
    df_LOS_Gexy_x_InSAR.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gexy_InSAR = df_LOS_Gexy_x_InSAR + df_LOS_Gexy_y_InSAR

    df_LOS_Gezz_InSAR = df_ezz_stacked_InSAR.loc[:,['Pz']] * df_pz
    

# SAVE G-matrix
# df_G_FT_on_InSAR_hori = [(exx1_x*px + exx1_y*py), ... , (exyHowManyCell_x*px + exyHowManyCell_y*py)]
    df_G_FT_on_InSAR_hori["GexxXPx+GexxYPy "+str(i)] = df_LOS_Gexx_InSAR.loc[:,['Py']]
    df_G_FT_on_InSAR_hori["GeyyXPx+GeyyYPy "+str(i)] = df_LOS_Geyy_InSAR.loc[:,['Py']]
    df_G_FT_on_InSAR_hori["GexyXPx+GexyYPy "+str(i)] = df_LOS_Gexy_InSAR.loc[:,['Py']]
    df_G_FT_on_InSAR_vert["GezzZPz "+str(i)] = df_LOS_Gezz_InSAR['Pz']

    
##################################################################################################################################
##################################################################################################################################
######################################################   Continuous Field   ######################################################    
##################################################################################################################################
##################################################################################################################################
    


    inputfile_exx_continuous = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_1"+"_continuous.gmt" #exx horizontal
    inputfile_eyy_continuous = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_2"+"_continuous.gmt" #eyy horizontal
    inputfile_exy_continuous = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_3"+"_continuous.gmt" #exy horizontal 
    inputfile_ezz_continuous = "./basis_functions_FT/vel_hori_FT_"+str(i)+"_4"+"_continuous.gmt" # z  horizontal   
    inputfile_zzz_continuous = "./basis_functions_FT/vel_vert_FT_"+str(i)+"_4"+"_continuous.gmt" # z  vertical
    
    
    df_exx_continuous=pd.read_csv(inputfile_exx_continuous, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy_continuous=pd.read_csv(inputfile_eyy_continuous, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy_continuous=pd.read_csv(inputfile_exy_continuous, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz_continuous=pd.read_csv(inputfile_ezz_continuous, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')
    df_zzz_continuous=pd.read_csv(inputfile_zzz_continuous, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')
    
# CHANGE the column names 

    df_exx_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_eyy_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_exy_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_ezz_continuous.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zzz_continuous.columns = ['lon','lat','vz']
    
    
# BUILD a column vector Gexx (i)

    df_exx_x_continuous = df_exx_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function
    df_exx_y_continuous = df_exx_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_exx_x_continuous=df_exx_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_exx_y_continuous=df_exx_y_continuous.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_x_continuous=df_exx_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_y_continuous=df_exx_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx_continuous=[df_exx_x_continuous,df_exx_y_continuous]
    df_Gexx_continuous=pd.concat(frames_Gexx_continuous,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Geyy (i)

    df_eyy_x_continuous = df_eyy_continuous.iloc[:,[0,1,2]]  # saved CONTINUOS vx basis function
    df_eyy_y_continuous = df_eyy_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_eyy_x_continuous=df_eyy_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_eyy_y_continuous=df_eyy_y_continuous.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_x_continuous=df_eyy_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_y_continuous=df_eyy_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Geyy_continuous=[df_eyy_x_continuous,df_eyy_y_continuous]
    df_Geyy_continuous=pd.concat(frames_Geyy_continuous,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gexy (i)
    df_exy_x_continuous = df_exy_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function
    df_exy_y_continuous = df_exy_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_exy_x_continuous=df_exy_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_exy_y_continuous=df_exy_y_continuous.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_x_continuous=df_exy_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_y_continuous=df_exy_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gexy_continuous=[df_exy_x_continuous,df_exy_y_continuous]
    df_Gexy_continuous=pd.concat(frames_Gexy_continuous,ignore_index=True) # merge the two dataFrames into one


# BUILD a column vector G_ezz (i)
    df_ezz_x_continuous = df_ezz_continuous.iloc[:,[0,1,2]]  # saved CONTINUOUS vx basis function
    df_ezz_y_continuous = df_ezz_continuous.iloc[:,[0,1,3]]  # saved CONTINUOUS vn basis function

    df_ezz_x_continuous=df_ezz_x_continuous.rename(columns ={'ve': 'velo'}) #column name change
    df_ezz_y_continuous=df_ezz_y_continuous.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_x_continuous=df_ezz_x_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_y_continuous=df_ezz_y_continuous.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gezz_continuous=[df_ezz_x_continuous,df_ezz_y_continuous]
    df_Gezz_continuous=pd.concat(frames_Gezz_continuous,ignore_index=True) # merge the two dataFrames into one (vertically)
    
# BUILD a column vector G_zzz (i)   
    df_zzz_continuous=df_zzz_continuous.sort_values(['lat','lon'], ascending=[True, True])
    df_zzz_continuous=df_zzz_continuous.reset_index(drop=True)
# SAVE a part of G-matrix (as in two different structures and then they will be merged later)

    # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowManyCell) Geyy(HowManyCell) Gexy(HowManyCell)]   
    df_G_FT_continuous_eij["G_exx"+str(i)] = df_Gexx_continuous.loc[:,['velo']]
    df_G_FT_continuous_eij["G_eyy"+str(i)] = df_Geyy_continuous.loc[:,['velo']]
    df_G_FT_continuous_eij["G_exy"+str(i)] = df_Gexy_continuous.loc[:,['velo']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowManyCell)] 
    df_G_FT_continuous_ezz["G_ezz"+str(i)] = df_Gezz_continuous.loc[:,['velo']]
    df_G_FT_continuous_zzz["G_zzz"+str(i)] = df_zzz_continuous.loc[:,['vz']]
    

    
#########STRAIN ###########
    inputfile_strain_exx = "./basis_functions_FT/average_strain_FT_"+str(i)+"_1_RECTANGULAR.out" #exx strain 
    inputfile_strain_eyy = "./basis_functions_FT/average_strain_FT_"+str(i)+"_2_RECTANGULAR.out" #eyy strain 
    inputfile_strain_exy = "./basis_functions_FT/average_strain_FT_"+str(i)+"_3_RECTANGULAR.out" #exy strain 
    inputfile_strain_ezz = "./basis_functions_FT/average_strain_FT_"+str(i)+"_4_RECTANGULAR.out" #ezz strain 

    
    df_exx_strain=pd.read_csv(inputfile_strain_exx, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy_strain=pd.read_csv(inputfile_strain_eyy, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy_strain=pd.read_csv(inputfile_strain_exy, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz_strain=pd.read_csv(inputfile_strain_ezz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')
    
    df_exx_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    df_eyy_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    df_exy_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    df_ezz_strain.columns = ['num','lat','lon','exx','eyy','exy','sxx','syy','sxy']
    
    # BUILD a column vector Gexx (i) : STRAIN
    df_exx_exx = df_exx_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints
    df_exx_eyy = df_exx_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints
    df_exx_exy = df_exx_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints
    df_exx_exx=df_exx_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_exx_eyy=df_exx_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_exx_exy=df_exx_exy.rename(columns ={'exy': 'strain'}) #column name change  
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_exx=df_exx_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_eyy=df_exx_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_exy=df_exx_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx_strain=[df_exx_exx,df_exx_eyy,df_exx_exy]
    df_Gexx_strain=pd.concat(frames_Gexx_strain,ignore_index=True) # merge the three dataFrames into one
    
    # BUILD a column vector Geyy (i) : STRAIN
    df_eyy_exx = df_eyy_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints
    df_eyy_eyy = df_eyy_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints
    df_eyy_exy = df_eyy_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints
    df_eyy_exx=df_eyy_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_eyy_eyy=df_eyy_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_eyy_exy=df_eyy_exy.rename(columns ={'exy': 'strain'}) #column name change  
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_exx=df_eyy_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_eyy=df_eyy_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_exy=df_eyy_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Geyy_strain=[df_eyy_exx,df_eyy_eyy,df_eyy_exy]
    df_Geyy_strain=pd.concat(frames_Geyy_strain,ignore_index=True) # merge the three dataFrames into one    
    

    # BUILD a column vector Gexy (i) : STRAIN
    df_exy_exx = df_exy_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints
    df_exy_eyy = df_exy_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints
    df_exy_exy = df_exy_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints
    df_exy_exx=df_exy_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_exy_eyy=df_exy_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_exy_exy=df_exy_exy.rename(columns ={'exy': 'strain'}) #column name change  
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_exx=df_exy_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_eyy=df_exy_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_exy=df_exy_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexy_strain=[df_exy_exx,df_exy_eyy,df_exy_exy]
    df_Gexy_strain=pd.concat(frames_Gexy_strain,ignore_index=True) # merge the three dataFrames into one   
    

    # BUILD a column vector Gezz (i) : STRAIN
    df_ezz_exx = df_ezz_strain.iloc[:,[2,1,3]]  # saved exx basis function on the midpoints
    df_ezz_eyy = df_ezz_strain.iloc[:,[2,1,4]]  # saved eyy basis function on the midpoints
    df_ezz_exy = df_ezz_strain.iloc[:,[2,1,5]]  # saved exy basis function on the midpoints
    df_ezz_exx=df_ezz_exx.rename(columns ={'exx': 'strain'}) #column name change
    df_ezz_eyy=df_ezz_eyy.rename(columns ={'eyy': 'strain'}) #column name change
    df_ezz_exy=df_ezz_exy.rename(columns ={'exy': 'strain'}) #column name change  
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_exx=df_ezz_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_eyy=df_ezz_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_exy=df_ezz_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gezz_strain=[df_ezz_exx,df_ezz_eyy,df_ezz_exy]
    df_Gezz_strain=pd.concat(frames_Gezz_strain,ignore_index=True) # merge the three dataFrames into one     
    
    

    df_G_FT_on_midpoints_eij["G_exx"+str(i)] = df_Gexx_strain.loc[:,['strain']]
    df_G_FT_on_midpoints_eij["G_eyy"+str(i)] = df_Geyy_strain.loc[:,['strain']]
    df_G_FT_on_midpoints_eij["G_exy"+str(i)] = df_Gexy_strain.loc[:,['strain']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowManyCell)] 
    df_G_FT_on_midpoints_ezz["G_ezz"+str(i)] = df_Gezz_strain.loc[:,['strain']]
    

    
# GNSS: Merge the two structures horizontally !
frames_Geij_Gezz = [df_G_FT_on_GNSS_eij, df_G_FT_on_GNSS_ezz]
df_G_FT_on_GNSS=pd.concat(frames_Geij_Gezz, axis=1) # merge the two dataFrames into one

# BC: Make a blank G matrix part related to Force Terms for Boundary Conditions == zeros
df_G_FT_on_boundary = pd.DataFrame(np.zeros((len(df_data_BC),HowManyCell*4)), columns=df_G_FT_on_GNSS.columns) 

# InSAR: Merge the two structures horizontally !
frames_FT_InSAR=[df_G_FT_on_InSAR_hori, df_G_FT_on_InSAR_vert]
df_G_FT_on_InSAR = pd.concat(frames_FT_InSAR, axis=1)  

# Continuous velocity Field (regular_lat_lon): Merge the two structures horizontally !
frames_Geij_Gezz = [df_G_FT_continuous_eij, df_G_FT_continuous_ezz]
df_G_FT_continuous=pd.concat(frames_Geij_Gezz, axis=1) # merge the two dataFrames into one

# Strain Field (Midpoints): Merge the two structures horizontally ! FOR strain
frames_Geij_Gezz_strain = [df_G_FT_on_midpoints_eij, df_G_FT_on_midpoints_ezz]
df_G_FT_on_midpoints_strain=pd.concat(frames_Geij_Gezz_strain, axis=1) # merge the two dataFrames into one  

/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2548728735.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_G_FT_on_GNSS_eij["G_eyy"+str(i)] = df_Geyy_GNSS.loc[:,['velo']]
/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2548728735.py:167: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_G_FT_on_GNSS_eij["G_exy"+str(i)] = df_Gexy_GNSS.loc[:,['velo']]
/var/folders/kt/zdsy5jqj2tn2lbh6qhjxdlgh0000gn/T/ipykernel_1192/2548728735.py:278: PerformanceWarning: DataFrame is highly fragmented.  This

In [10]:
print("DONE")

DONE


### `STEP 2-c :` Build the complete G matrix for the inversion

In [11]:
# GNSS part FT and BC
frames_FT_BC_on_GNSS = [df_G_FT_on_GNSS, df_G_BC_on_GNSS]
df_G_FT_BC_on_GNSS = pd.concat(frames_FT_BC_on_GNSS, axis=1) #merge the two DataFrames into one DF horizontally (axis = 1)
 
# InSAR part FT and BC
frames_FT_BC_on_InSAR = [df_G_FT_on_InSAR, df_G_BC_on_InSAR]
df_G_FT_BC_on_InSAR = pd.concat(frames_FT_BC_on_InSAR, axis=1) #merge the two DataFrames into one DF horizontally (axis = 1)

# boundary part FT and BC
frame_FT_BC_on_boundary = [df_G_FT_on_boundary, df_G_BC_on_boundary]
df_G_FT_BC_on_boundary = pd.concat(frame_FT_BC_on_boundary, axis=1) #merge the two DataFrames into one DF horizontally (axis = 1)

##################################################################################################################################

df_G_FT_BC_on_boundary.columns = df_G_FT_BC_on_InSAR.columns
df_G_FT_BC_on_GNSS.columns = df_G_FT_BC_on_InSAR.columns
# change column names of the df_G_FT_BC_on_boundary
# change column names of the df_G_FT_BC_on_GNSS

frames_FT_BC_final=[df_G_FT_BC_on_GNSS, df_G_FT_BC_on_InSAR, df_G_FT_BC_on_boundary]
df_G_final = pd.concat(frames_FT_BC_final, ignore_index=True) #merge the two dataFrames into one vertically (axis = 1)


In [12]:
if len(df_data_total)!=len(df_G_final):
    print("WARNING: Something went wrong!")

In [15]:
# SAVE G-matrix for the inversion
df_G_final.to_csv(outputFILE_G_matrix, index=None, float_format='%g')

<div class="alert alert-success">
    <b> Save G-matrix for the inversion. </b>
</div>

In [23]:
########################################################################################
#                         Horizontal continuous velocity model.                        #
########################################################################################
frames_horizontal = [df_G_FT_continuous, df_G_BC_continuous]
df_G_horizontal_continuous=pd.concat(frames_horizontal, axis=1) # merge the two dataFrames into one

########################################################################################
#                         Vertical continuous velocity model.                          #
########################################################################################
df_G_vertical_continuous=df_G_FT_continuous_zzz
########################################################################################
#                         Horizontal continuous strain rate model.                     #
########################################################################################
frames_horizontal_strain = [df_G_FT_on_midpoints_strain, df_G_BC_on_midpoints_strain]
df_G_horizontal_continuous_strain=pd.concat(frames_horizontal_strain, axis=1) # merge the two dataFrames into one


In [24]:
# SAVE G-matrices for the 3-D motion field & strain field predictions
#df_G_horizontal_continuous.to_csv(outputFILE_hori_continuous, index=None, float_format='%g')
df_G_vertical_continuous.to_csv(outputFILE_vert_continuous, index=None, float_format='%g')
df_G_horizontal_continuous_strain.to_csv(outputFILE_strain, index=None, float_format='%g')

<div class="alert alert-success">
    <b> Save G-matrices for the predictions. </b>
</div>